## Import libraries

In [1]:
# Importing the library
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
from IPython.display import display # Manage multiple output per cell
import matplotlib.pyplot as plt # plotting library
import datetime
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold

/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Configuration

In [2]:
odd_H = 'INFO_BbAvH'
odd_A = 'INFO_BbAvA'
odd_D = 'INFO_BbAvD'
target = 'INFO_FTR'
start_date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
season_list = [2014, 2015, 2016]
#season_list = [2016]
league = 'SP1'
classes = ['A', 'D', 'H']

In [3]:
best_features_MLP = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS','H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HS', 'H_MEANS_THREE_AS','A_STD_FIVE_HF', 'H_STD_FIVE_HC', 'H_STD_FIVE_HST']
all_features = ["A_MEANS_FIVE_AC","A_MEANS_FIVE_AF","A_MEANS_FIVE_AR","A_MEANS_FIVE_AS","A_MEANS_FIVE_AST","A_MEANS_FIVE_AY","A_MEANS_FIVE_FTAG","A_MEANS_FIVE_FTHG","A_MEANS_FIVE_FTR_A","A_MEANS_FIVE_FTR_D","A_MEANS_FIVE_FTR_H","A_MEANS_FIVE_HC","A_MEANS_FIVE_HF","A_MEANS_FIVE_HR","A_MEANS_FIVE_HS","A_MEANS_FIVE_HST","A_MEANS_FIVE_HTAG","A_MEANS_FIVE_HTHG","A_MEANS_FIVE_HTR_A","A_MEANS_FIVE_HTR_D","A_MEANS_FIVE_HTR_H","A_MEANS_FIVE_HY","H_MEANS_FIVE_AC","H_MEANS_FIVE_AF","H_MEANS_FIVE_AR","H_MEANS_FIVE_AS","H_MEANS_FIVE_AST","H_MEANS_FIVE_AY","H_MEANS_FIVE_FTAG","H_MEANS_FIVE_FTHG","H_MEANS_FIVE_FTR_A","H_MEANS_FIVE_FTR_D","H_MEANS_FIVE_FTR_H","H_MEANS_FIVE_HC","H_MEANS_FIVE_HF","H_MEANS_FIVE_HR","H_MEANS_FIVE_HS","H_MEANS_FIVE_HST","H_MEANS_FIVE_HTAG","H_MEANS_FIVE_HTHG","H_MEANS_FIVE_HTR_A","H_MEANS_FIVE_HTR_D","H_MEANS_FIVE_HTR_H","H_MEANS_FIVE_HY","A_MEANS_THREE_AC","A_MEANS_THREE_AF","A_MEANS_THREE_AR","A_MEANS_THREE_AS","A_MEANS_THREE_AST","A_MEANS_THREE_AY","A_MEANS_THREE_FTAG","A_MEANS_THREE_FTHG","A_MEANS_THREE_FTR_A","A_MEANS_THREE_FTR_D","A_MEANS_THREE_FTR_H","A_MEANS_THREE_HC","A_MEANS_THREE_HF","A_MEANS_THREE_HR","A_MEANS_THREE_HS","A_MEANS_THREE_HST","A_MEANS_THREE_HTAG","A_MEANS_THREE_HTHG","A_MEANS_THREE_HTR_A","A_MEANS_THREE_HTR_D","A_MEANS_THREE_HTR_H","A_MEANS_THREE_HY","H_MEANS_THREE_AC","H_MEANS_THREE_AF","H_MEANS_THREE_AR","H_MEANS_THREE_AS","H_MEANS_THREE_AST","H_MEANS_THREE_AY","H_MEANS_THREE_FTAG","H_MEANS_THREE_FTHG","H_MEANS_THREE_FTR_A","H_MEANS_THREE_FTR_D","H_MEANS_THREE_FTR_H","H_MEANS_THREE_HC","H_MEANS_THREE_HF","H_MEANS_THREE_HR","H_MEANS_THREE_HS","H_MEANS_THREE_HST","H_MEANS_THREE_HTAG","H_MEANS_THREE_HTHG","H_MEANS_THREE_HTR_A","H_MEANS_THREE_HTR_D","H_MEANS_THREE_HTR_H","H_MEANS_THREE_HY","A_STD_FIVE_AC","A_STD_FIVE_AF","A_STD_FIVE_AR","A_STD_FIVE_AS","A_STD_FIVE_AST","A_STD_FIVE_AY","A_STD_FIVE_FTAG","A_STD_FIVE_FTHG","A_STD_FIVE_FTR_A","A_STD_FIVE_FTR_D","A_STD_FIVE_FTR_H","A_STD_FIVE_HC","A_STD_FIVE_HF","A_STD_FIVE_HR","A_STD_FIVE_HS","A_STD_FIVE_HST","A_STD_FIVE_HTAG","A_STD_FIVE_HTHG","A_STD_FIVE_HTR_A","A_STD_FIVE_HTR_D","A_STD_FIVE_HTR_H","A_STD_FIVE_HY","H_STD_FIVE_AC","H_STD_FIVE_AF","H_STD_FIVE_AR","H_STD_FIVE_AS","H_STD_FIVE_AST","H_STD_FIVE_AY","H_STD_FIVE_FTAG","H_STD_FIVE_FTHG","H_STD_FIVE_FTR_A","H_STD_FIVE_FTR_D","H_STD_FIVE_FTR_H","H_STD_FIVE_HC","H_STD_FIVE_HF","H_STD_FIVE_HR","H_STD_FIVE_HS","H_STD_FIVE_HST","H_STD_FIVE_HTAG","H_STD_FIVE_HTHG","H_STD_FIVE_HTR_A","H_STD_FIVE_HTR_D","H_STD_FIVE_HTR_H","H_STD_FIVE_HY","A_STD_THREE_AC","A_STD_THREE_AF","A_STD_THREE_AR","A_STD_THREE_AS","A_STD_THREE_AST","A_STD_THREE_AY","A_STD_THREE_FTAG","A_STD_THREE_FTHG","A_STD_THREE_FTR_A","A_STD_THREE_FTR_D","A_STD_THREE_FTR_H","A_STD_THREE_HC","A_STD_THREE_HF","A_STD_THREE_HR","A_STD_THREE_HS","A_STD_THREE_HST","A_STD_THREE_HTAG","A_STD_THREE_HTHG","A_STD_THREE_HTR_A","A_STD_THREE_HTR_D","A_STD_THREE_HTR_H","A_STD_THREE_HY","H_STD_THREE_AC","H_STD_THREE_AF","H_STD_THREE_AR","H_STD_THREE_AS","H_STD_THREE_AST","H_STD_THREE_AY","H_STD_THREE_FTAG","H_STD_THREE_FTHG","H_STD_THREE_FTR_A","H_STD_THREE_FTR_D","H_STD_THREE_FTR_H","H_STD_THREE_HC","H_STD_THREE_HF","H_STD_THREE_HR","H_STD_THREE_HS","H_STD_THREE_HST","H_STD_THREE_HTAG","H_STD_THREE_HTHG","H_STD_THREE_HTR_A","H_STD_THREE_HTR_D","H_STD_THREE_HTR_H","H_STD_THREE_HY"]
best_features_NB = ['A_MEANS_FIVE_AC', 'A_MEANS_FIVE_AS', 'A_MEANS_FIVE_AST','A_MEANS_FIVE_FTAG', 'A_MEANS_FIVE_FTHG', 'A_MEANS_FIVE_FTR_H','A_MEANS_FIVE_HC', 'A_MEANS_FIVE_HS', 'A_MEANS_FIVE_HST','A_MEANS_FIVE_HTR_A', 'H_MEANS_FIVE_AC', 'H_MEANS_FIVE_AS','H_MEANS_FIVE_AST', 'H_MEANS_FIVE_AY', 'H_MEANS_FIVE_FTAG','H_MEANS_FIVE_FTHG', 'H_MEANS_FIVE_FTR_A', 'H_MEANS_FIVE_FTR_H','H_MEANS_FIVE_HC', 'H_MEANS_FIVE_HS', 'H_MEANS_FIVE_HST','H_MEANS_FIVE_HTR_H', 'A_MEANS_THREE_AC', 'A_MEANS_THREE_AS','A_MEANS_THREE_FTHG', 'A_MEANS_THREE_HS', 'H_MEANS_THREE_AS','A_STD_FIVE_HF', 'H_STD_FIVE_HC', 'H_STD_FIVE_HST']
features_list = [
    ['best_features_MLP', best_features_MLP],
    ['all_features', all_features],
    ['best_features_NB', best_features_NB]
]

In [4]:
# Parameter for layer 1
best_params = {
    'C': 8.291,
    'penalty': 'l2',
    'class_weight': None,
    'solver': 'sag',
    'max_iter': 270,
    'multi_class': 'multinomial'
}

In [5]:
df_result = pd.DataFrame()

In [6]:
# Construct base layer
base_layer = [
    ['XGBoost', False, 'no', 9, XGBClassifier(
        learning_rate=0.01,
        n_estimators=160,
        max_depth=8,
        min_child_weight=7,
        gamma=0.28,
        subsample=0.8,
        colsample_bytree=0.6,
        objective='multi:softprob',
        reg_alpha=0.87,
        nthread=4,
        scale_pos_weight=1,
        seed=15), 
     ['all_features', all_features]
    ],
    ['NB', True, 'no', 9, GaussianNB(), ['best_features_NB', best_features_NB]],
    ['MLP', True, 'no', 9, MLPClassifier(
        random_state=0,
        activation='logistic', 
        alpha=0.3, 
        hidden_layer_sizes=(60,),
        max_iter=190, 
        solver='sgd'),
     ['best_features_MLP', best_features_MLP]
    ],
]

In [7]:
# Configure number of fold
NFOLDS = 4

## Import Data

In [8]:
# DB Sqlite connection
import sqlite3
db = "/Users/thibaultclement/Project/ligue1-predict/src/notebook/data/db/soccer_predict.sqlite"
conn = sqlite3.connect(db)
cur = conn.cursor()

In [9]:
# Get all prematch data
df_all = pd.read_sql_query("SELECT * FROM pre_matchs ORDER BY INFO_Date ASC;", conn)
df_all = (df_all[df_all.columns.drop(['index'])])
df_all.shape

(37907, 190)

In [10]:
# Remove all game between June (include) and October (include)
df_all['INFO_Date'] = pd.to_datetime(df_all['INFO_Date'])
df_all['INFO_Date'].dt.month
df_all = df_all[(df_all['INFO_Date'].dt.month < 6) | (df_all['INFO_Date'].dt.month >= 10)]
df_all.shape

(30912, 190)

In [11]:
# Create a INFO_WIN column containing the gain if you bet the good result
df_all['INFO_WIN'] = 0
df_all.loc[df_all.INFO_FTR == 'H', 'INFO_WIN'] = df_all[odd_H]
df_all.loc[df_all.INFO_FTR == 'A', 'INFO_WIN'] = df_all[odd_A]
df_all.loc[df_all.INFO_FTR == 'D', 'INFO_WIN'] = df_all[odd_D]
df_all['INFO_WIN_P'] = 0
df_all.loc[df_all.INFO_FTR == 'H', 'INFO_WIN_P'] = df_all['INFO_PSH']
df_all.loc[df_all.INFO_FTR == 'A', 'INFO_WIN_P'] = df_all['INFO_PSA']
df_all.loc[df_all.INFO_FTR == 'D', 'INFO_WIN_P'] = df_all['INFO_PSD']

## Methods

In [12]:
def get_dataset(league, season, preprocessing, historical_training_year, features):
    # Filter by league
    df = df_all[(df_all['INFO_Div'] == league)]
    # Keep season for test and filter by number of historical season used to train
    date_start_learn = datetime.date(season-historical_training_year, 8, 1)
    date_end_learn = datetime.date(season, 8, 1)
    date_start_test_season = datetime.date(season, 8, 1)
    date_end_test_season = datetime.date(season+1, 8, 1)
    df_test = df[(df['INFO_Date'] > date_start_test_season)]
    df_test = df_test[(df_test['INFO_Date'] < date_end_test_season)]
    df_test = df_test[(df_test['INFO_Date'].dt.month < 6) | (df_test['INFO_Date'].dt.month > 10)]
    df = df[(df['INFO_Date'] > date_start_learn)]
    df = df[(df['INFO_Date'] < date_end_learn)]
    # reset index
    df = df.reset_index()
    df_test = df_test.reset_index()
    # Filter by feature used to train
    X = pd.get_dummies(df[features])
    y = df[target]
    X_test_season = pd.get_dummies(df_test[features])
    y_test_season = df_test[target]
    # Impute of missing values (NaN) with the mean
    # TODO drop NaN instead of replacing ith means 
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    imp = imp.fit(X)
    X = imp.transform(X)
    X_test_season = imp.transform(X_test_season)
    # Standardize features
    if preprocessing:
        sc_X = StandardScaler().fit(X)
        X = sc_X.transform(X)
        X_test_season = sc_X.transform(X_test_season)
    return df, df_test, X, y, X_test_season, y_test_season

In [13]:
# Out of fold prediction
def get_oof(clf, x_train, y_train, x_test):
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    kf = KFold(ntrain, n_folds=NFOLDS, shuffle=True, random_state=0)

    oof_train = np.zeros((x_train.shape[0],3))
    oof_test = np.zeros((x_test.shape[0],3))
    oof_test_skf = np.empty((NFOLDS, x_test.shape[0], 3))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        # Calibrate model
        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict_proba(x_te)
        oof_test_skf[i, :] = clf.predict_proba(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train, oof_test

In [14]:

def get_layer_columns(layer, classes):
    cols = []
    for clf_name, preprocessing, calibration, historical_training_year, clf, features in layer:
        for result in classes:
            cols.append(clf_name+result)
    return cols

In [15]:
def get_layer1_df(X, X_test_season, base_layer, cols):
    X_train_layer1 = np.zeros((X.shape[0], len(base_layer)*3))
    X_train_layer1 = pd.DataFrame(X_train_layer1, columns=cols)
    X_test_layer1 = np.zeros((X_test_season.shape[0], len(base_layer)*3))
    X_test_layer1 = pd.DataFrame(X_test_layer1, columns=cols)
    return X_train_layer1, X_test_layer1

In [16]:
def get_score(y, pred, probs):
    # Compute cross-entropy score
    ll = log_loss(y, probs)
    # Compute accuracy score
    acc = accuracy_score(y, pred)
    # Compute precision score
    prec = precision_score(y, pred, average=None)
    prec_A = prec[0]
    prec_D = prec[1]
    prec_H = prec[2]
    # Compute recall score
    rec = recall_score(y, pred, average=None)
    rec_A = rec[0]
    rec_D = rec[1]
    rec_H = rec[2]
    # Compute F1 score
    f1 = f1_score(y, pred, average=None)
    f1_A = f1[0]
    f1_D = f1[1]
    f1_H = f1[2]
    return {
        'll': ll, 
        'acc': acc, 
        'prec_A': prec_A, 
        'prec_D': prec_D, 
        'prec_H': prec_H, 
        'rec_A': rec_A, 
        'rec_D': rec_D, 
        'rec_H': rec_H, 
        'f1_A': f1_A, 
        'f1_D': f1_D, 
        'f1_H': f1_H
    }

In [17]:
def get_money(df_test, pred_season, prob_season):
    # Join odd and prediction together
    df_test_season = df_test
    df_test_season['probs_A'] = prob_season[:,0]
    df_test_season['probs_D'] = prob_season[:,1]
    df_test_season['probs_H'] = prob_season[:,2]
    df_test_season['probs'] = df_test_season[['probs_A','probs_D','probs_H']].max(axis=1)
    #df_test_season['pred'] = le.inverse_transform(pred_season)
    df_test_season['pred'] = pred_season
    df_test_season['WIN'] = -1
    df_test_season.loc[df_test_season.INFO_FTR == df_test_season.pred, 'WIN'] = df_test_season['INFO_WIN']-1
    df_test_season['WIN_P'] = -1
    df_test_season.loc[df_test_season.INFO_FTR == df_test_season.pred, 'WIN_P'] = df_test_season['INFO_WIN_P']-1
    df_test_season['INFO_ODD_BET'] = 0
    df_test_season.loc[df_test_season.pred == 'A', 'INFO_ODD_BET'] = df_test_season[odd_A]
    df_test_season.loc[df_test_season.pred == 'D', 'INFO_ODD_BET'] = df_test_season[odd_D]
    df_test_season.loc[df_test_season.pred == 'H', 'INFO_ODD_BET'] = df_test_season[odd_H]
    df_test_season['prob_less_bet'] = 0
    df_test_season.loc[df_test_season.pred == 'A', 'prob_less_bet'] = df_test_season['probs'] - df_test_season[odd_A].apply(lambda x: 1/x)
    df_test_season.loc[df_test_season.pred == 'D', 'prob_less_bet'] = df_test_season['probs'] - df_test_season[odd_D].apply(lambda x: 1/x)
    df_test_season.loc[df_test_season.pred == 'H', 'prob_less_bet'] = df_test_season['probs'] - df_test_season[odd_H].apply(lambda x: 1/x)
    # calculate money I can get following different scenario
    # Bet on all
    bet_all = df_test_season.WIN.mean()
    # Bet under 1.9
    bet_lte_19 = df_test_season[df_test_season['INFO_ODD_BET'] < 1.9].WIN.mean()
    # Bet under 4
    bet_lte_4 = df_test_season[df_test_season['INFO_ODD_BET'] < 4].WIN.mean()
    # Bet between 1.9 and 4
    bet_btw_19_4 = df_test_season[(df_test_season['INFO_ODD_BET'] > 1.9) & (df_test_season['INFO_ODD_BET'] < 4)].WIN.mean()
    # Bet between 1.9 and 5
    bet_btw_19_5 = df_test_season[(df_test_season['INFO_ODD_BET'] > 1.9) & (df_test_season['INFO_ODD_BET'] < 5)].WIN.mean()
    # Bet between 1.5 and 4
    bet_btw_15_4 = df_test_season[(df_test_season['INFO_ODD_BET'] > 1.5) & (df_test_season['INFO_ODD_BET'] < 4)].WIN.mean()
    # Bet between 1.5 and 5
    bet_btw_15_5 = df_test_season[(df_test_season['INFO_ODD_BET'] > 1.5) & (df_test_season['INFO_ODD_BET'] < 5)].WIN.mean()
    # Bet prob higher than 50%
    bet_pred_gte_50 = df_test_season[df_test_season.probs > 0.5].WIN.mean()
    # Bet prob higher than 60%
    bet_pred_gte_60 = df_test_season[df_test_season.probs > 0.6].WIN.mean()
    # Bet prob higher than 70%
    bet_pred_gte_70 = df_test_season[df_test_season.probs > 0.7].WIN.mean()
    return {
        'bet_all': bet_all,
        'bet_lte_19': bet_lte_19,
        'bet_lte_4': bet_lte_4,
        'bet_btw_19_4': bet_btw_19_4,
        'bet_btw_19_5': bet_btw_19_5,
        'bet_btw_15_4': bet_btw_15_4,
        'bet_btw_15_5': bet_btw_15_5
    }


## Loop on season

In [18]:
# Init dataframe
result_df = pd.DataFrame(columns=[
    'league', 
    'season',
    'C',
    'penalty',
    'class_weight',
    'solver',
    'max_iter',
    'multi_class',
    'll',
    'acc',
    'prec_A',
    'prec_D',
    'prec_H',
    'rec_A',
    'rec_D',
    'rec_H',
    'f1_A',
    'f1_D',
    'f1_H',
    'bet_all',
    'bet_lte_19',
    'bet_lte_4',
    'bet_btw_19_4',
    'bet_btw_19_5',
    'bet_btw_15_4',
    'bet_btw_15_5'
])

In [19]:
for season in season_list:
    print league,str(season)
    
    # Prepare the layer 1
    df, df_test, X, y, X_test_season, y_test_season = get_dataset(league, season, True, 9, all_features)
    cols = get_layer_columns(base_layer, classes)
    X_layer1, X_layer1_test_season = get_layer1_df(X, X_test_season, base_layer, cols)
    
    # train base layer
    for clf_name, preprocessing, calibration, historical_training_year, classifier, features in base_layer:
        
        #Get the dataset
        df, df_test, X, y, X_test_season, y_test_season = get_dataset(league, season, preprocessing, historical_training_year, features[1])
        
        print "Processing model:",clf_name
        # Check if we need to recalibrate the prediction
        if calibration == 'sigmoid':
            clf = CalibratedClassifierCV(classifier, cv=4, method='sigmoid')
        elif calibration == 'isotonic':
            clf = CalibratedClassifierCV(classifier, cv=4, method='isotonic')
        elif calibration == 'no':
            clf = classifier
        # obtain out-of-fold predictions for this model
        oof_train, oof_test = get_oof(clf, X, y, X_test_season)
        X_layer1.loc[:, [clf_name+result for result in classes]] = oof_train
        
        # Base Layer for test season
        #Get the dataset
        df2, df_test, X2, y2, X_test_season, y_test_season = get_dataset(league, season, preprocessing, historical_training_year, features[1])
        clf_test = classifier
        clf_test.fit(X, y)
        predict_probs_test = clf_test.predict_proba(X_test_season)
        X_layer1_test_season.loc[:, [clf_name+result for result in classes]] = predict_probs_test

    # train stacking model
    clf_1 = LogisticRegression()
    clf_1.fit(X_layer1, y)
    # Predict target values
    y_pred = clf_1.predict(X_layer1_test_season)
    # Predict probabilities
    y_probs = clf_1.predict_proba(X_layer1_test_season)
    # get scores
    score_dict = get_score(y_test_season, y_pred, y_probs)
    # get money earned
    money_dict = get_money(df_test, y_pred, y_probs)
    # Keep result for further analyis
    df_result = df_result.append(df_test)
    # Add all info to result dataframe
    result_df.loc[len(result_df.index)] = [
        league, 
        season, 
        best_params['C'],
        best_params['penalty'],
        best_params['class_weight'],
        best_params['solver'],
        best_params['max_iter'],
        best_params['multi_class'],
        score_dict['ll'],
        score_dict['acc'],
        score_dict['prec_A'],
        score_dict['prec_D'],
        score_dict['prec_H'],
        score_dict['rec_A'],
        score_dict['rec_D'],
        score_dict['rec_H'],
        score_dict['f1_A'],
        score_dict['f1_D'],
        score_dict['f1_H'],
        money_dict['bet_all'],
        money_dict['bet_lte_19'],
        money_dict['bet_lte_4'],
        money_dict['bet_btw_19_4'],
        money_dict['bet_btw_19_5'],
        money_dict['bet_btw_15_4'],
        money_dict['bet_btw_15_5']
    ]

SP1 2016
Processing model: XGBoost
Processing model: NB
Processing model: MLP


/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Save result

In [20]:
#result_df.to_csv('./report/STACKING_1_FTR_E0_BEST_VALIDATION.csv')

In [21]:
df_result.to_csv('../report/SP1-last-3-seasons-oct-7.csv')

### Final result
Best is with ??? and ??? years of history

In [22]:
# Remove games I didn't bet on
df_bet_current_season = df_result
#df_bet_current_season = df_bet_current_season.drop(df_bet_current_season[df_bet_current_season.A_MEANS_FIVE_FTHG < 1].index)
#df_bet_current_season = df_bet_current_season.drop(df_bet_current_season[df_bet_current_season.A_MEANS_FIVE_FTHG > 3].index)
#df_bet_current_season = df_bet_current_season.drop(df_bet_current_season[df_bet_current_season.A_MEANS_FIVE_FTAG < 1].index)
#df_bet_current_season = df_bet_current_season.drop(df_bet_current_season[df_bet_current_season.A_MEANS_FIVE_FTAG > 3].index)
#df_bet_current_season = df_bet_current_season.drop(df_bet_current_season[df_bet_current_season.probs <= 0.4].index)
#df_bet_current_season = df_bet_current_season.drop(df_bet_current_season[df_bet_current_season.pred != 'H'].index)
#df_bet_current_season = df_bet_current_season.drop(df_bet_current_season[df_bet_current_season.pred == 'D'].index)
#df_bet_current_season = df_bet_current_season.drop(df_bet_current_season[df_bet_current_season.prob_less_bet <= 0].index)
#df_bet_current_season = df_bet_current_season[df_bet_current_season.prob_less_bet > 0]
#df_bet_current_season = df_bet_current_season[df_bet_current_season['INFO_ODD_BET'] > 2]
#df_bet_current_season = df_bet_current_season[df_bet_current_season['INFO_ODD_BET'] < 4]

In [23]:
# What will I bet on
display(plt.show(), 100. * df_bet_current_season.pred.value_counts() / len(df_bet_current_season.pred))

None

H    79.642857
A    20.357143
Name: pred, dtype: float64

In [24]:
# What will be the real result of games I bet on
display(plt.show(), 100. * df_bet_current_season.INFO_FTR.value_counts() / len(df_bet_current_season.INFO_FTR))

None

H    49.285714
A    28.571429
D    22.142857
Name: INFO_FTR, dtype: float64

In [25]:
df_bet_current_season.WIN.mean()

0.05260714285714292

In [26]:
df_bet_current_season.WIN_P.mean()

0.0727857142857143

In [27]:
df_bet_current_season.shape

(280, 202)

In [28]:
df_bet_current_season[['INFO_Date', 'probs_A','probs_D','probs_H', 'probs', 'prob_less_bet', 'pred', 'INFO_FTR', 'WIN']]

,INFO_Date,probs_A,probs_D,probs_H,probs,prob_less_bet,pred,INFO_FTR,WIN
0,2016-11-04,0.167953,0.227795,0.604252,0.604252,0.019456,H,H,0.71
1,2016-11-05,0.223692,0.280891,0.495417,0.495417,0.116629,H,D,-1.00
2,2016-11-05,0.277670,0.242678,0.479652,0.479652,-0.030552,H,H,0.96
3,2016-11-05,0.245409,0.294012,0.460579,0.460579,0.018101,H,A,-1.00
4,2016-11-05,0.279699,0.242891,0.477410,0.477410,0.282478,H,H,4.13
5,2016-11-06,0.263743,0.250151,0.486105,0.486105,0.043627,H,H,1.26
6,2016-11-06,0.285772,0.308046,0.406182,0.406182,0.058960,H,D,-1.00
7,2016-11-06,0.164339,0.175527,0.660134,0.660134,-0.265792,H,H,0.08
8,2016-11-06,0.428740,0.180930,0.390329,0.428740,-0.181016,A,A,0.64
9,2016-11-06,0.196499,0.201387,0.602115,0.602115,-0.055780,H,H,0.52
